In [ ]:
import pandas as pd
import numpy as np
import censusdata

from sklearn.model_selection import train_test_split
import seaborn as sns

## Exploring ACS Data 

This notebook is an initial exploration of ACS data accessed through the Python library CensusData which is built on the Census API.

In [ ]:
##search for specific variable name based on indicator to grab later
##indicators available at https://www.census.gov/programs-surveys/acs/guidance/subjects.html

#uncomment to view
censusdata.censustable('acs1', 2015, 'B25002')

### ETL

In [ ]:
def metric_by_year_by_county(census_table_code_list, year, acs):
    data = censusdata.download(acs, year, censusdata.censusgeo([('state', '*'), ('county', '*')]),
          census_table_code_list).reset_index()
    data['state'] = data['index'].apply(lambda x: str(x).split(',')[1].split(':')[0].strip())
    data['county'] = data['index'].apply(lambda x: str(x).split('County')[0].strip())
    data['fips'] = data['index'].apply(lambda x: int(str(x).split(':')[-2][:2].strip() + str(x).split(':')[-1].strip()))
    data.drop('index', axis=1, inplace=True)
    return data

In [ ]:
def df_creation():
    df = pd.DataFrame()

    for i in range(2015, 2020, 1):
        metric_df = metric_by_year_by_county(['B25002_002E', 'B25002_003E', 'B25031_003E', 'B08303_001E',
                             'B25031_004E', 'B25031_005E', 'B25031_006E', 'B25077_001E', 'B01003_001E',
                             'B13016_003E', 'B13016_004E', 'B08201_001E', 'B19001_001E',
                             'B13016_005E', 'B13016_006E', 'B13016_007E', 'B13016_008E', 'B13016_009E',
                             'B13016_011E', 'B13016_012E', 'B13016_013E', 'B13016_014E', 'B15003_001E',
                             'B13016_015E', 'B13016_016E', 'B13016_017E', 'B15003_025E',
                             'B15003_022E', 'B15003_023E', 'B15003_024E'], i, 'acs1')
            
        metric_df['year'] = i
        df = df.append(metric_df)
    return df            

In [ ]:
#dictionary to maps codes to indicator names

indicator_dict = {'B01003_001E': 'population', 'B08303_001E': 'travel_time_to_work', 
                  'B08201_001E': 'vehicles_available', 'B19001_001E': 'household_income',
                 'B25077_001E': 'home_value_median', 'B15003_022E': 'bachelor_degree',
                 'B25002_002E': 'occupied_units', 'B25002_003E': 'vacant_units',
                 'B25031_003E': 'rent_1bed_median', 'B15003_023E': 'master_degree',
                 'B25031_004E': 'rent_2bed_median', 'B25031_005E': 'rent_3bed_median',
                 'B25031_006E': 'rent_4bed_median', 'B15003_024E': 'professional_degree',
                 'B13016_003E': 'birth_15_19', 'B15003_025E': 'doctorate_degree',
                 'B13016_004E': 'birth_20_24', 'B13016_005E': 'birth_25_29', 'B13016_006E': 'birth_30_34',
                 'B13016_007E': 'birth_35_39', 'B13016_008E': 'birth_40_44', 'B13016_009E': 'birth_45_50',
                 'B13016_011E': 'nobirth_15_19', 'B13016_012E': 'nobirth_20_24', 'B13016_013E': 'nobirth_25_29',
                 'B13016_014E': 'nobirth_30_34', 'B13016_015E': 'nobirth_35_39', 'B13016_016E': 'nobirth_40_44',
                 'B13016_017E': 'nobirth_45_50'}

In [ ]:
df = df_creation()

df.rename(columns=indicator_dict,
          inplace=True)
df.replace(-666666666, np.nan, inplace=True)

In [ ]:
for col in df.columns:
    print(col+':', df[col].isnull().sum(), 'null values')

In [ ]:
df['year'].unique()

In [ ]:
#fill nulls and create ratios

for col in df.columns:
    try:
        mean = df[col].mean()
        df[col].fillna(value=mean, inplace=True)
    except:
        continue

df['birth_ratio_15_19'] = df['birth_15_19']/df['nobirth_15_19']
df['birth_ratio_20_24'] = df['birth_20_24']/df['nobirth_20_24']
df['birth_ratio_25_29'] = df['birth_25_29']/df['nobirth_25_29']
df['birth_ratio_30_34'] = df['birth_30_34']/df['nobirth_30_34']
df['birth_ratio_35_39'] = df['birth_35_39']/df['nobirth_35_39']
df['birth_ratio_40_44'] = df['birth_40_44']/df['nobirth_40_44']
df['birth_ratio_45_50'] = df['birth_45_50']/df['nobirth_45_50']

df['bachelor_ratio'] = df['bachelor_degree']/df['population']
df['master_ratio'] = df['master_degree']/df['population']
df['professional_ratio'] = df['professional_degree']/df['population']
df['doctorate_ratio'] = df['doctorate_degree']/df['population']

df['occupied_ratio'] = df['occupied_units']/(df['occupied_units'] + df['vacant_units'])


In [ ]:
df.shape

In [ ]:
#bring in HPI data

hpi_df = pd.read_excel('HPI_AT_BDL_county (1).xlsx', header=6)

In [ ]:
hpi_df.head()

In [ ]:
hpi_df = hpi_df[hpi_df['Year']>2015]

hpi_df = hpi_df[['Year', 'FIPS code', 'Annual Change (%)']]

hpi_df.columns = ['year', 'fips', 'hpi_change_in_a_year']

In [ ]:
#reduce year by 1 so that the hpi annual change value is for one year in the future
hpi_df['year'] = hpi_df['year'].apply(lambda x: x-1)

In [ ]:
#merge with indicators and remove nan (not allowed in model)
df = df.merge(hpi_df, how='left', on=['year', 'fips'])

df.replace('.', np.nan, inplace=True)
df['hpi_change_in_a_year'] = df['hpi_change_in_a_year'].astype(float)


df = df.dropna()

## Modeling

In [ ]:
# train test split
# try using historical population for population change
X = df[['rent_1bed_median',
       'travel_time_to_work', 'rent_2bed_median', 'rent_3bed_median',
       'rent_4bed_median', 'home_value_median', 'population', 'birth_15_19',
       'birth_20_24', 'vehicles_available', 'household_income', 'birth_ratio_15_19', 
        'birth_ratio_20_24', 'birth_ratio_25_29',
       'birth_ratio_30_34', 'birth_ratio_35_39', 'birth_ratio_40_44',
       'birth_ratio_45_50', 'bachelor_ratio', 'master_ratio',
       'professional_ratio', 'doctorate_ratio', 'occupied_ratio']]
y = df['hpi_change_in_a_year']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(X_train, y_train)

In [ ]:
regr.score(X_test, y_test)

In [ ]:
regr.feature_importances_

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(df[['rent_1bed_median',
       'travel_time_to_work', 'rent_2bed_median', 'rent_3bed_median',
       'rent_4bed_median', 'home_value_median', 'population', 'birth_15_19',
       'birth_20_24', 'vehicles_available', 'household_income', 'birth_ratio_15_19', 
        'birth_ratio_20_24', 'birth_ratio_25_29',
       'birth_ratio_30_34', 'birth_ratio_35_39', 'birth_ratio_40_44',
       'birth_ratio_45_50', 'bachelor_ratio', 'master_ratio',
       'professional_ratio', 'doctorate_ratio', 'occupied_ratio', 'hpi_change_in_a_year']].corr())